In [ ]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np

In [ ]:
# pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df = nfl.import_pbp_data([2024])

In [ ]:
df

In [ ]:
df.loc[df.home_team=='BUF']

In [ ]:
game = df.loc[df['game_id']=="2024_20_BAL_BUF"].copy()

In [ ]:
game

In [ ]:
game.info()

In [ ]:
game.columns.tolist()

In [ ]:
game.tail()

In [ ]:
game.loc[df.play_id==1]

In [ ]:
game.loc[[48253,48254], ['play_id','posteam','time','down','ydstogo','yrdln','play_type','desc','ep', 'epa','total_home_epa','total_away_epa']]

In [ ]:
game.loc[[48405], ['play_id','time','down','ydstogo','yrdln','play_type','desc','ep', 'epa','total_home_epa','total_away_epa']]

In [ ]:
game.tail(20)['desc']

In [ ]:
game.loc[[48403], ['play_id','time','down','ydstogo','yrdln','play_type','desc','ep', 'epa','total_home_epa','total_away_epa']]

In [ ]:
game.loc[48409, ['total_home_epa', 'total_home_score','total_away_epa', 'total_away_score']]

In [ ]:
game.groupby('posteam')['epa'].agg('sum')

In [ ]:
game[['time','down','ydstogo','yrdln','posteam','epa']]

In [ ]:
game.loc[game['posteam']=='BAL', ['time','down','ydstogo','yrdln','posteam','epa','total_home_epa']]

In [ ]:
game.groupby(['posteam','play_type']).agg({'epa': 'sum'})

In [ ]:
game.loc[game['play_type']=='qb_kneel', ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep', 'epa']]

In [ ]:
game.loc[game['play_type']=='punt', ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep', 'epa']]

In [ ]:
game.loc[game['drive']==4, ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa']]

In [ ]:
game.groupby(['posteam', 'down']).agg({'ydstogo': 'mean'})

In [ ]:
game.groupby(['posteam', 'down']).agg({'epa': 'mean'})

In [ ]:
game.groupby('drive').agg({'epa': 'sum'})

In [ ]:
game.loc[game['drive']==1, ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa']]

In [ ]:
game.loc[game['play_type']=='kickoff', ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa']]

In [ ]:
game.loc[game['desc']=='END GAME', ['home_team','total_home_score','total_home_epa','total_home_rush_epa','total_home_pass_epa','away_team','total_away_score','total_away_epa','total_away_rush_epa','total_away_pass_epa']]

In [ ]:
game.loc[game['play_type']=='run', ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa','total_home_epa','total_away_epa']]

In [ ]:
game.loc[game['drive']==4, ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa','total_home_epa','total_away_epa']]

In [ ]:
# need to find drives with kickoffs and/or penalties, see if that explains the difference between summing epa and the total epa column
# in fact, let's go through each drive one at a time to see where the divergence starts

In [ ]:
game['drive'].describe()

In [ ]:
drive = game.loc[game['drive']==3, ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa','total_home_epa','total_away_epa']]
drive

In [ ]:
drive['epa'].sum()

In [ ]:
drive['total_home_epa'].tail(1) - game.loc[drive.index[0]-1, 'total_home_epa']

In [ ]:
# okay, they never disagree, so i must have been summing epa wrong before. let's try again.

In [ ]:
game.groupby('posteam').agg({'epa': 'sum'})

In [ ]:
# what am i missing?

In [ ]:
for d in range(1,19):
    drive = game.loc[game['drive']==d, ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa','total_home_epa','total_away_epa']]
    print(drive['epa'].sum())

In [ ]:
total_epa = {"BUF": 0, "BAL": 0}
total_epa

In [ ]:
for d in range(1,19):
    drive = game.loc[game['drive']==d, ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa','total_home_epa','total_away_epa']]
    total_epa[drive['posteam'].values[0]] += drive['epa'].sum()

In [ ]:
total_epa

In [ ]:
total_epa['BAL'] - total_epa['BUF']

In [ ]:
# that's the exact value of total_away_epa....

In [ ]:
game[['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa','total_home_epa','total_away_epa']]

In [ ]:
game.loc[48275:48279,['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa','total_home_epa','total_away_epa']]

In [ ]:
# i understand! each team's total epa is affected on all drives, not just ones where they're in possession.
# so i can just do what i did before, but without doing it by drive like that.

In [ ]:
total_epa = {game['home_team'].values[0]: 0, game['away_team'].values[0]: 0}
total_epa

In [ ]:
# testing apply
def add_epa(row):
    posteam = row['posteam']
    epa = row['epa']
    if posteam:
        total_epa[posteam] += epa
    return(total_epa)

game.apply(add_epa, axis=1)

In [ ]:
total_epa

In [ ]:
game.loc[game['posteam']=='BUF', ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa','total_home_epa','total_away_epa']]['epa']

In [ ]:
game.loc[game['posteam']=='BUF', ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa','total_home_epa','total_away_epa']]['epa'].values[-1]

In [ ]:
if game.loc[game['posteam']=='BUF', ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa','total_home_epa','total_away_epa']]['epa'].values[-1]:
    print('hello')

In [ ]:
game.loc[game['posteam']=='BUF', ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa','total_home_epa','total_away_epa']]['epa'].values[-1] == game.loc[game['posteam']=='BUF', ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa','total_home_epa','total_away_epa']]['epa'].values[-1]

In [ ]:
total_epa = {game['home_team'].values[0]: 0, game['away_team'].values[0]: 0}
total_epa

In [ ]:
def add_epa(row):
    posteam = row['posteam']
    defteam = row['defteam']
    epa = row['epa']
    if posteam and epa==epa:
        total_epa[posteam] += epa
        total_epa[defteam] -= epa
    return(total_epa)

game.apply(add_epa, axis=1)

In [ ]:
total_epa

In [ ]:
# now let's try a better version of that, hopefully without apply

In [ ]:
game.drop(game[game['posteam'].isna()].index, inplace=True)
game[game['posteam'].isna()]

In [ ]:
game.drop(game[game['epa'].isna()].index, inplace=True)
game[game['epa'].isna()]

In [ ]:
game['home_epa'] = np.where(game['posteam']==game['home_team'], game['epa'], -1*game['epa'])

In [ ]:
game[['home_epa', 'epa']]

In [ ]:
game['away_epa'] = np.where(game['posteam']==game['away_team'], game['epa'], -1*game['epa'])

In [ ]:
game[['away_epa','epa']]

In [ ]:
game['my_total_home_epa'] = game['home_epa'].cumsum()

In [ ]:
game[['my_total_home_epa', 'total_home_epa']]

In [ ]:
game['my_total_away_epa'] = game['away_epa'].cumsum()

In [ ]:
game[['my_total_away_epa', 'total_away_epa']]

In [ ]:
game['my_total_home_epa'] == game['total_home_epa']

In [ ]:
# must be tiny rounding errors. good enough for me!

In [ ]:
# and now i actually understand why the diff of the original sum seemed to give the actual total
# sum of all one teams poss' epa - the sum of the epa of their opp on the other poss. seems obvious now lol

In [ ]:
# so really i can do the original groupby instead of messing with all these excess columns

In [ ]:
grouped_epa = game.groupby('posteam').agg({'epa': 'sum'})
grouped_epa

In [ ]:
home_poss_epa = grouped_epa.loc[game['home_team'].values[0]].values[0]
home_poss_epa

In [ ]:
away_poss_epa = grouped_epa.loc[game['away_team'].values[0]].values[0]
away_poss_epa

In [ ]:
home_epa = home_poss_epa - away_poss_epa
home_epa

In [ ]:
away_epa = away_poss_epa - home_poss_epa
away_epa

In [ ]:
# once i confirm my own total epa calculations, i can confirm the rush and pass ones
# then i can ignore certain plays, like qb_kneel, and calculate special teams and penalties epa, etc.

In [ ]:
# once i've confirmed all that, let's create a new notebook dedicated to just that, with a function that spits out the final values of all those columns for each game
# then we can iterate over all the games and use that function, and have a dictionary of each team's epa for each game